<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/pruebas_CarlosKong/proyecto_final_APR_V02_en_colab_ck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MONTAJE EN GOOGLE DRIVE
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

import os
if IN_COLAB:
  print("Montando Google Drive en", mount)
  drive.mount(mount)
  os.makedirs(drive_root, exist_ok=True)
  %cd $drive_root
%pwd

# INSTALACIÓN DE DEPENDENCIAS
%pip install gym==0.17.3
%pip install git+https://github.com/Kojoley/atari-py.git
%pip install keras-rl2==1.0.5
%pip install tensorflow==2.12
%pip install opencv-python

# IMPORTACIONES
import numpy as np
import gym
import cv2
#import tensorflow as tf
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Montando Google Drive en /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-8rpezb7_
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-8rpezb7_
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


In [2]:
# ENTORNO
env = gym.make('SpaceInvaders-v0')  # Cambia si deseas: 'Breakout-v0'
nb_actions = env.action_space.n

# PREPROCESAMIENTO
class AtariProcessor(Processor):
    def process_observation(self, observation):
        img = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (84, 84))
        return img.astype('uint8')

    def process_state_batch(self, batch):
        return batch.astype('float32') / 255.0

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [3]:
# MODELO DQN
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(4, 84, 84)))
model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

# MEMORIA Y POLICY
memory = SequentialMemory(limit=100000, window_length=4)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.1, value_test=0.05,
                              nb_steps=200000)

In [4]:
# AGENTE DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,
               nb_steps_warmup=50000, enable_double_dqn=True,
               enable_dueling_network=True, dueling_type='avg',
               target_model_update=10000, policy=policy,
               processor=AtariProcessor())

from keras.optimizers import Adam
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

# CHECKPOINTS Y LOGGING
checkpoint_path = drive_root + '/dqn_weights_{step}.h5f'
weights_filename = drive_root + '/dqn_final_weights.h5f'

callbacks = [
    ModelIntervalCheckpoint(checkpoint_path, interval=25000),
    FileLogger(drive_root + '/dqn_log.json', interval=10000)
]

/usr/local/lib/python3.11/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
#dqn_weights_100000.h5f
#Continuación al 325000 + 100000 + 100000+75000+100000 = 800000

dqn.load_weights("dqn_weights_75000.h5f")
# 🔢 Paso de partida 3
starting_step = 75000 #ultimo weight actualizado
remaining_steps = 200000 - starting_step


# 🧩 Callback personalizado para continuar numeración de checkpoints
class OffsetModelCheckpoint(ModelIntervalCheckpoint):
    def __init__(self, filepath, interval, offset):
        super().__init__(filepath, interval)
        self.offset = offset

    def on_step_end(self, step, logs={}):
        # Ajusta el número de paso en el nombre del archivo
        self.step = step + self.offset
        super().on_step_end(step, logs)

# 🚀 Entrenamiento con pasos continuados desde 100000
dqn.fit(env, nb_steps=remaining_steps, visualize=False, verbose=2,
        callbacks=[
            FileLogger("dqn_log_continuacion.json", interval=10000),
            OffsetModelCheckpoint("dqn_weights_{step}.h5f", interval=25000, offset=starting_step)
        ])

Training for 125000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    634/125000: episode: 1, duration: 4.627s, episode steps: 634, steps per second: 137, episode reward: 11.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.475 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1406/125000: episode: 2, duration: 5.157s, episode steps: 772, steps per second: 150, episode reward:  9.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.547 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1807/125000: episode: 3, duration: 1.898s, episode steps: 401, steps per second: 211, episode reward:  5.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.546 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2589/125000: episode: 4, duration: 3.808s, episode steps: 782, steps per second: 205, episode reward: 12.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.386 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   3305/125000: episode: 5, duration: 5.100s, episode steps: 716, steps per 

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  50326/125000: episode: 79, duration: 81.127s, episode steps: 482, steps per second:   6, episode reward:  7.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.685 [0.000, 5.000],  loss: 0.011988, mae: 0.480977, mean_q: 0.591011, mean_eps: 0.774266
  50836/125000: episode: 80, duration: 122.197s, episode steps: 510, steps per second:   4, episode reward:  6.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.496 [0.000, 5.000],  loss: 0.011484, mae: 0.479145, mean_q: 0.585658, mean_eps: 0.772388
  51476/125000: episode: 81, duration: 156.931s, episode steps: 640, steps per second:   4, episode reward: 11.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.466 [0.000, 5.000],  loss: 0.010772, mae: 0.475709, mean_q: 0.580060, mean_eps: 0.769800
  51971/125000: episode: 82, duration: 119.482s, episode steps: 495, steps per second:   4, episode reward: 11.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 2.394 [0.000, 5.000],  loss: 0.009741, mae: 0.476042, mean_q

In [7]:

weights_filename = 'dqn_weights_125000.h5f'.format(env)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 9.000, steps: 520
Episode 2: reward: 22.000, steps: 980
Episode 3: reward: 10.000, steps: 601
Episode 4: reward: 22.000, steps: 953
Episode 5: reward: 21.000, steps: 1026
Episode 6: reward: 10.000, steps: 638
Episode 7: reward: 6.000, steps: 477
Episode 8: reward: 9.000, steps: 522
Episode 9: reward: 8.000, steps: 688
Episode 10: reward: 7.000, steps: 598


In [8]:
#dqn_weights_100000.h5f
#Continuación al 325000 + 100000 + 100000+75000+100000+125000 = 925000

dqn.load_weights("dqn_weights_125000.h5f")
# 🔢 Paso de partida 5
starting_step = 125000 #ultimo weight actualizado
remaining_steps = 225000 - starting_step


# 🧩 Callback personalizado para continuar numeración de checkpoints
class OffsetModelCheckpoint(ModelIntervalCheckpoint):
    def __init__(self, filepath, interval, offset):
        super().__init__(filepath, interval)
        self.offset = offset

    def on_step_end(self, step, logs={}):
        # Ajusta el número de paso en el nombre del archivo
        self.step = step + self.offset
        super().on_step_end(step, logs)

# 🚀 Entrenamiento con pasos continuados desde 100000
dqn.fit(env, nb_steps=remaining_steps, visualize=False, verbose=2,
        callbacks=[
            FileLogger("dqn_log_continuacion.json", interval=10000),
            OffsetModelCheckpoint("dqn_weights_{step}.h5f", interval=25000, offset=starting_step)
        ])

Training for 100000 steps ...
   609/100000: episode: 1, duration: 3.928s, episode steps: 609, steps per second: 155, episode reward:  3.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 2.621 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1483/100000: episode: 2, duration: 5.362s, episode steps: 874, steps per second: 163, episode reward: 14.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.543 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2082/100000: episode: 3, duration: 3.132s, episode steps: 599, steps per second: 191, episode reward:  8.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.459 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2745/100000: episode: 4, duration: 3.480s, episode steps: 663, steps per second: 191, episode reward: 10.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.511 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3492/100000: episode: 5, duration: 5.292s, episo